In [1]:
import cv2
import numpy as np
import math

def resize_image_to_1920x1080(image_path, output_path="resized_image.jpg"):
    """
    Resizes the image to 1920x1080 resolution.
    Args:
        image_path (str): Path to the input image.
        output_path (str): Path to save the resized image.
    Returns:
        np.array: Resized image.
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Unable to load image. Check the file path.")
    
    # Resize to 1920x1080
    resized_image = cv2.resize(image, (1920, 1080), interpolation=cv2.INTER_AREA)
    cv2.imwrite(output_path, resized_image)  # Save the resized image
    return resized_image

def calculate_diameter(image_path, resolution_width, resolution_height, height_cm, AOV_deg, is_circular=True):
    """
    Detects a circular or hexagonal object and calculates its diameter or equivalent measurement.
    Args:
        image_path (str): Path to the resized image (1920x1080).
        resolution_width (int): Image resolution width in pixels.
        resolution_height (int): Image resolution height in pixels.
        height_cm (float): Distance from the camera to the object (in cm).
        AOV_deg (float): Angle of View of the camera (in degrees).
        is_circular (bool): If True, detect a circular object; else, detect a hexagonal object.
    Returns:
        float: Diameter or equivalent measurement in cm.
    """
    # Calculate FOV width and height using height_cm and AOV_deg
    AOV_rad = math.radians(AOV_deg)
    FOV_width_cm = 2 * height_cm * math.tan(AOV_rad / 2)
    
    # Calculate the aspect ratio
    aspect_ratio = resolution_width / resolution_height
    
    # Calculate the FOV height based on the aspect ratio
    FOV_height_cm = FOV_width_cm / aspect_ratio
    
    # Calculate Pixels per cm
    pixels_per_cm_width = resolution_width / FOV_width_cm

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Unable to load image. Check the file path.")
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Thresholding
    _, thresh = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour (assuming it’s the object)
    largest_contour = max(contours, key=cv2.contourArea)

    if is_circular:
        # Fit a minimum enclosing circle
        ((x, y), radius) = cv2.minEnclosingCircle(largest_contour)
        object_pixels = 2 * radius  # Diameter in pixels
    else:
        # Fit a bounding rectangle and use the width for approximation
        x, y, w, h = cv2.boundingRect(largest_contour)
        object_pixels = max(w, h)  # Use the larger side for hexagonal approximation

    # Calculate the real-world diameter in cm
    object_diameter_cm = object_pixels / pixels_per_cm_width

    # Draw contour and circle on the image for visualization
    output_image = image.copy()
    if is_circular:
        cv2.circle(output_image, (int(x), int(y)), int(radius), (0, 255, 0), 2)
    else:
        cv2.rectangle(output_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.imwrite("detected_object.jpg", output_image)

    return object_diameter_cm

# Example Usage
if __name__ == "__main__":
    try:
        # Step 1: Take inputs
        image_path = input("Enter the path to the image: ").strip()
        resolution_width = int(input("Enter the image resolution width (in pixels): "))
        resolution_height = int(input("Enter the image resolution height (in pixels): "))
        height_cm = float(input("Enter the height of the camera from the object (in cm): "))
        AOV_deg = float(input("Enter the Angle of View (in degrees): "))
        object_type = input("Enter the object type (circular/hexagonal): ").strip().lower()

        # Validate object type
        if object_type not in ["circular", "hexagonal"]:
            raise ValueError("Invalid object type. Please enter 'circular' or 'hexagonal'.")

        # Step 2: Resize the image
        resized_image = resize_image_to_1920x1080(image_path)

        # Step 3: Calculate the diameter
        is_circular = object_type == "circular"
        diameter_cm = calculate_diameter("resized_image.jpg", resolution_width, resolution_height, height_cm, AOV_deg, is_circular)

        # Output the result
        print(f"The {object_type} object's diameter is approximately {diameter_cm:.2f} cm.")
    except ValueError as e:
        print(f"Error: {e}")


The circular object's diameter is approximately 7.96 cm.
